# スクワット角度データ分析Notebook

In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

In [12]:
# csvのインポート　ダンベルスクワットは除外
# csvは各々が出力したものを入力する

Full = pd.read_csv("../output/フルスクワット.csv").iloc[280:670].reset_index(drop=True)
# Dumbell = pd.read_csv("../output/ダンベルスクワット.csv").iloc[60:680].reset_index(drop=True)
Half = pd.read_csv("../output/ハーフスクワット.csv")
TomPlatz = pd.read_csv("../output/TPスクワット.csv").iloc[210:600].reset_index(drop=True)

Full["class"] = "Full"
# Dumbell["class"] = "Dumbell"
Half["class"] = "Half"
TomPlatz["class"] = "TomPlatz"

In [ ]:
# 波形データを確認
plt.plot(Full["大腿角度R"])
# plt.xlabel('Time')
plt.ylabel('angle')
plt.title('Sample ECG-like Waveform')
plt.show()

In [ ]:
def calcu_peak(df,col_name,dist = 50):
    """
    波形データの上限値と下限値を抽出する関数
    """
    peaks, _ = find_peaks(df[col_name], distance=dist)
    inverted_peaks, _ = find_peaks(-df[col_name], distance=dist)

    # 各ピークの位置と値を表示する
    for peak in peaks:
        print("Peak Time:", df.index[peak], "Peak Value:", df[col_name][peaks])

    # 各最小値の位置と値を表示する
    for inverted_peak in inverted_peaks:
        print("Inverted Peak Time:", df.index[inverted_peak], "Inverted Peak Value:", df[col_name][inverted_peak])

    # 各ピークと最小値をプロットする
    plt.plot(df.index, df[col_name])
    plt.plot(df.index[peaks], df[col_name][peaks], 'ro')  # ピークを赤色の点で表示する
    plt.plot(df.index[inverted_peaks], df[col_name][inverted_peaks], 'go')  # 最小値を緑色の点で表示する
    plt.xlabel('Time')
    plt.ylabel(f'{col_name}')
    plt.show()

    return df[[col_name,"class"]].iloc[peaks,:] , df[[col_name,"class"]].iloc[inverted_peaks,:]


In [ ]:

df_full_peak , df_full_invers = calcu_peak(Full,"膝角度R",dist = 45)
# df_dumbl_peak , df_dumbl_invers = calcu_peak(Dumbell,"上体角度R")
df_half_peak , df_half_invers = calcu_peak(Half,"膝角度R",dist = 80)
df_tom_peak , df_tom_invers = calcu_peak(TomPlatz,"膝角度R",dist = 45)


In [ ]:
# 膝角度を結合
df_hiza_peak = pd.concat([df_full_peak,df_half_peak,df_tom_peak])
df_hiza_invers = pd.concat([df_full_invers,df_half_invers,df_tom_invers])

df_hiza_peak["peak"] = "upper"
df_hiza_invers["peak"] = "lower"

## 大腿角度

In [ ]:

df_full_peak , df_full_invers = calcu_peak(Full,"大腿角度R",dist = 45)
# df_dumbl_peak , df_dumbl_invers = calcu_peak(Dumbell,"大腿角度R")
df_half_peak , df_half_invers = calcu_peak(Half,"大腿角度R",dist = 80)
df_tom_peak , df_tom_invers = calcu_peak(TomPlatz,"大腿角度R",dist = 40)


In [ ]:
df_daitai_peak = pd.concat([df_full_peak,df_half_peak,df_tom_peak])
df_daitai_invers = pd.concat([df_full_invers,df_half_invers,df_tom_invers])

df_daitai_peak["peak"] = "upper"
df_daitai_invers["peak"] = "lower"

In [ ]:
import seaborn as sns

df_cat = pd.concat([df_daitai_peak,df_daitai_invers])
markers = {'upper': 'X', 'lower': 'o'}
df_cat['marker'] = df_cat['peak'].map(markers)

plt.figure(figsize=(10, 8))
sns.scatterplot(data=df_cat, x='class', y='大腿角度R', hue='class', style='marker', s=200, palette='Set1',legend=None)
# plt.xlabel('class')
# plt.ylabel('大腿角度R')
plt.tick_params(axis='x', labelsize=20)
plt.tick_params(axis='y', labelsize=20)
plt.show()

In [ ]:
import seaborn as sns

df_cat2 = pd.concat([df_hiza_peak,df_hiza_invers])
markers = {'upper': 'X', 'lower': 'o'}
df_cat2['marker'] = df_cat2['peak'].map(markers)

plt.figure(figsize=(10, 8))
sns.scatterplot(data=df_cat2, x='class', y='膝角度R', hue='class', style='marker', s=200, palette='Set1',legend=None)
plt.tick_params(axis='x', labelsize=20)
plt.tick_params(axis='y', labelsize=20)
plt.show()

# 大腿四頭筋長さと大腿角度の比較

In [ ]:

fig = plt.figure(figsize=(15, 8))

# plt.figure()
ax1 = fig.add_subplot(111)
fs = 1.0
y1 = Full["大腿角度R"]
ln1=ax1.plot(y1,'C0')

ax2 = ax1.twinx()
y2 = Full["大腿直筋肉長さR"]
ln2=ax2.plot(y2,'C1')

# ax1.set_ylabel('hoge')
ax1.grid(True)
